Давайте вернёмся к данным выживаемости пациентов с лейкоцитарной лимфомой из видео про критерий знаков:

49, 58, 75, 110, 112, 132, 151, 276, 281, 362^*49,58,75,110,112,132,151,276,281,362 
∗
 

Измерено остаточное время жизни с момента начала наблюдения (в неделях); звёздочка обозначает цензурирование сверху — исследование длилось 7 лет, и остаточное время жизни одного пациента, который дожил до конца наблюдения, неизвестно.

Поскольку цензурировано только одно наблюдение, для проверки гипотезы H0:medX=200 на этих данных можно использовать критерий знаковых рангов — можно считать, что время дожития последнего пациента в точности равно 362, на ранг этого наблюдения это никак не повлияет.

Критерием знаковых рангов проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости, округлённый до четырёх знаков после десятичной точки.

In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [4]:
data = [49,58,75,110,112,132,151,276,281,362]
m0 = 200
stats.wilcoxon(np.array(data) - m0)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

In [98]:
f1 = [22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15]
f2 = [17,18,18,15,12,4,14,15,10]
f1 = np.array(f1)
f2 = np.array(f2)

In [99]:
stats.mannwhitneyu(f1, f2, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.02900499272087373)

С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после запятой.

Чтобы получить в точности такой же доверительный интервал, как у нас:

установите random seed = 0 перед первым вызовом функции get_bootstrap_samples, один раз
сделайте по 1000 псевдовыборок из каждой выборки.

In [16]:
challenger = pd.read_table('challenger.txt', sep = '\t', header = 0)
challenger.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [20]:
no_damage=challenger[challenger.Incident == 0].Temperature.values
damage=challenger[challenger.Incident == 1].Temperature.values
print no_damage, '\n',  damage

[18.9 20.6 20.  19.4 22.2 22.8 21.1 25.6 19.4 19.4 23.9 21.1 27.2 24.4
 26.1 24.4] 
[21.1 13.9 17.2 21.1 11.7 23.9 14.4]


In [28]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries


In [22]:
np.random.seed(0)

In [23]:
damage_scores = map(np.mean, get_bootstrap_samples(damage, 1000))
no_damage_scores = map(np.mean, get_bootstrap_samples(no_damage, 1000))

print "95% confidence interval for the damage mean:",  stat_intervals(damage_scores, 0.05)
print "95% confidence interval for the no damage mean:",  stat_intervals(no_damage_scores, 0.05)


95% confidence interval for the damage mean: [14.61428571 20.71607143]
95% confidence interval for the no damage mean: [21.14359375 23.55046875]


In [24]:
delta_median_scores = map(lambda x: x[1] - x[0], zip(damage_scores, no_damage_scores))


In [27]:
print "95% confidence interval for the difference between medians", \
    stat_intervals(delta_median_scores, 0.05)

95% confidence interval for the difference between medians [1.45040179 8.06457589]


In [80]:
np.array(n1)[[2,0]]

array([22, 11])

In [69]:
from itertools import permutations 
from sympy.utilities.iterables import multiset_permutations
n1 = [11, 22, 22]
n2 = [44, 555]

In [83]:
# x = set([tuple(sorted(x)) for x in permutations(n1 + n2, len(n1))])
# x
from itertools import combinations 
x = list(combinations(range(len(n1)+len(n2)), len(n1)))
x

[(0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 2, 3),
 (0, 2, 4),
 (0, 3, 4),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 4),
 (2, 3, 4)]

In [97]:
get_random_combinations([11, 22, 22], [22, 77], 1000)

AttributeError: 'builtin_function_or_method' object has no attribute 'choices'

In [90]:
import random

In [96]:
def get_random_combinations(sample1, sample2, max_combinations):
    
    n1 = len(sample1)
    n2 = len(sample2)
    s = np.array(sample1 + sample2)
    x = list(combinations(range(n1+n2), n1))
    ind_1 = random.choice(x,)
    ind_2 = [set(range(n1+n2)) - set(i) for i in ind_1]
    return [s[i1] for i1 in ind_1], [s[i2] for i2 in ind_2]

In [29]:
def permutation_zero_distr_1sample(sample, mean, max_permutations = None):
    centered_sample = map(lambda x: x - mean, sample)
    if max_permutations:
        signs_array = set([tuple(x) for x in 2 * np.random.randint(2, size = (max_permutations, 
                                                                              len(sample))) - 1 ])
    else:
        signs_array =  itertools.product([-1, 1], repeat = len(sample))
    distr = [sum(centered_sample * np.array(signs)) for signs in signs_array]
    return distr

In [36]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_test(sample1, sample2, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample1, sample2)
    
#     zero_distr = get_random_combinations(sample, mean, max_permutations)
    zero_distr = get_random_combinations(len(sample1), len(sample2), max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [35]:
print "p-value: %f" % permutation_test(damage, no_damage, 0., 
                               max_permutations = 50000)

TypeError: bad operand type for abs(): 'tuple'